## Run Experiments

### Import libraries

In [1]:
from benchmarks.experiments.wrapper_overhead.common.run_utility import WrapperOverheadRunUtility
import re

### Setup Common Resources

In [2]:
region = 'us-east-2'
run_utility: WrapperOverheadRunUtility = WrapperOverheadRunUtility(region)

times = 1

In [3]:
import re

# Example log entry
log_entry = ("[INFO] 2024-03-21T19:01:44.224Z 69a4c93b-7dd5-4c84-bd1f-16ff6f50b87e Workload Name: workflow-wo-dna_vis-ed-direct_sns, "
             "Request ID: 758c5963-12a6-4501-aa43-9b9fa96a282e, Client Start Time: 2024-03-21 19:00:11,367321+0000, "
             "First Function Start Time: 2024-03-21 19:00:08,606980+0000, Time Taken from workload invocation from client: 92857.007 ms, "
             "Time Taken from first function: 95617.348 ms, Function End Time: 2024-03-21 19:01:44,224328+0000")

# Regular expression to extract the relevant parts of the log entry
pattern = re.compile(r"Workload Name: (.*?), Request ID: (.*?), Client Start Time: (.*?), "
                     r"First Function Start Time: (.*?), Time Taken from workload invocation from client: (.*?) ms, "
                     r"Time Taken from first function: (.*?) ms, Function End Time: (.*)")

# Parsing the log entry
match = pattern.search(log_entry)
if match:
    workload_name, req_id, start_time_str, first_function_start_time_str, ms_from_start, ms_from_first_function, final_function_end_time = match.groups()
    print(f"Workload Name: {workload_name}\nRequest ID: {req_id}\nClient Start Time: {start_time_str}\n"
          f"First Function Start Time: {first_function_start_time_str}\nTime Taken from workload invocation from client: {ms_from_start} ms\n"
          f"Time Taken from first function: {ms_from_first_function} ms\nFunction End Time: {final_function_end_time}")
else:
    print("No match found")


Workload Name: workflow-wo-dna_vis-ed-direct_sns
Request ID: 758c5963-12a6-4501-aa43-9b9fa96a282e
Client Start Time: 2024-03-21 19:00:11,367321+0000
First Function Start Time: 2024-03-21 19:00:08,606980+0000
Time Taken from workload invocation from client: 92857.007 ms
Time Taken from first function: 95617.348 ms
Function End Time: 2024-03-21 19:01:44,224328+0000


## Perform Runs for each benchmarks

#### Boto3 Direct Calls

In [4]:
dna_vis_direct_calls_dir = "./dna_visualization/external_database/boto3_only_direct_calls" # Boto3 only direct calls
run_utility.run_experiment(dna_vis_direct_calls_dir, payload, times)

Running boto3_direct workload: workflow-wo-dna_vis-ed-direct
Payload: {'gen_file_name': 'small_sequence.gb'}
Times: 1
Done



#### Boto3 SNS calls

In [5]:
dna_vis_sns_calls_dir = "./dna_visualization/external_database/boto3_only_sns" # Boto3 only sns calls
run_utility.run_experiment(dna_vis_sns_calls_dir, payload, times)

Running boto3_sns workload: workflow-wo-dna_vis-ed-direct_sns
Payload: {'gen_file_name': 'small_sequence.gb', 'metadata': {'workload_name': 'workflow-wo-dna_vis-ed-direct', 'experiment_type': 'boto3_direct', 'request_id': '6fe96778-7cd8-426d-bd72-4852ba4daa02', 'start_time': '2024-03-21 19:00:10,374521+0000'}}
Times: 1
Done



#### AWS Step Function

In [6]:
dna_vis_aws_step_functions_dir = "./dna_visualization/external_database/aws_step_function" # AWS Step Functions
run_utility.run_experiment(dna_vis_aws_step_functions_dir, payload, times)

Running aws_step_function workload: wo-dna_vis-ed-step_function
Payload: {'gen_file_name': 'small_sequence.gb', 'metadata': {'workload_name': 'workflow-wo-dna_vis-ed-direct_sns', 'experiment_type': 'boto3_sns', 'request_id': '758c5963-12a6-4501-aa43-9b9fa96a282e', 'start_time': '2024-03-21 19:00:11,367321+0000'}}
Times: 1


#### Multi-X

In [5]:
dna_vis_direct_calls_dir = "./dna_visualization/external_database/multi_x" # Boto3 only direct calls
run_utility.run_experiment(dna_vis_direct_calls_dir, payload, times)

Running multi_x workload: wo-dna_vis-ed-multi_x-0.0.2
Payload: {'gen_file_name': 'small_sequence.gb', 'metadata': {'workload_name': 'wo-dna_vis-ed-multi_x-0.0.2', 'experiment_type': 'multi_x', 'request_id': '258e50ee-eb35-450e-a6ec-c9edbd0ce454', 'start_time': '2024-03-22 15:44:14,133436+0000'}}
Times: 1
Sending to home region: False, time_key: N/A
Done

